Reference:  
https://zhuanlan.zhihu.com/p/42310942  NLP中的文本表示方法  
https://zhuanlan.zhihu.com/p/56382372  nlp中的词向量对比：word2vec/glove/fastText/elmo/GPT/bert  
http://www.hankcs.com/nlp/word2vec.html#respond  word2vec原理推导与代码分析

## NLP中的文本表示
- 基于ONE-HOT, TF-IDF, TEXTRANK等的BAG-OF-WORDS
- 主题模型: LSA(SVD), pLSA, LDA;
- 基于词向量的固定表征: word2vec, fastText, glove
- 基于词向量的动态表征: elmo, GPT, bert
### 离散表示
##### One-hot
词的表示(文本中的一个基元)给每个词编码一个索引,根据索引对每个词进行ont-hot表示.  
*特点*: 数据稀疏,维度灾难, 无法表达单词与单词之间的相似程度, 无法表达一词多义的情况
##### BAG-OF-WORDS
也叫做计数向量表示, 一种文档表示方法: 忽略文档的单词顺序和语法,句法等要素, 将其仅仅看作是若干个词汇的集合,且文档中每个词的出现都是相互独立的.
描述: 有一个文档集合D，里面一共有M个文档，而文档里面的所有单词提取出来后，一起构成一个包含N个单词的词典，利用Bag-of-words模型，每个文档都可以被表示成为一个N维向量，向量中的每个元素表示对应的单词在文档中出现的次数. 这样，就可以利用计算机来完成海量文档的分类过程。

##### Bi-gram和N-gram
(文本/文档/句子)与词袋模型类似, 将相邻的两个或者N个词编上索引.  
*特点*: 引入了词的顺序, 造成词向量的急剧膨胀, N一般取3 最多不超过5
##### TF-IDF
对语料库中的一个文档进行表示: 需要统计一个文档中所有词语在语料库中的逆文档频率. 得到文档中每个词语的逆文档频率. (一般会去除停用词)  
*特点*: 使用词语的各个单词频率描述一篇文档.  
*用途*: 抽取文档关键词. 优点是简单快速, 结果也比较符合实际情况; 缺点是无法体现词的位置信息, 仅以词频衡量词的重要性也不够全面.  
*sklearn中的TfidfVectorizer生成TF-IDF特征*:
```python
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
```
##### 共现矩阵(Cocurrence matrix)
如两个词在一句话或者一篇文章中共同出现. 通常我们选择一个距离窗口, 如果窗口宽度为2则认为当前词前后两个词的距离范围内出现的词为共现. 扫描所有文本,统计每个单词的共现词构造共现矩阵(对角矩阵). 对共现矩阵进行PCA或者SVD操作进行降维.  
*特点*: 使用稠密向量,计算量大

### 分布式表示
离散表示虽然能够进行词语或者文本的表示, 进而用于情感分析或者文本分类的人物. 但其不能表示词语间的相似程度或者词语间的类比关系. 因此使用分布式表示, 其主要思想是: **一个词的含义可以用它周围的词进行表示, 因为相同上下文语境的词往往会有相似的含义**  
#### 静态表征
##### NNLM
主要目的是为了搭建模型用于*根据前N-1个词预测第N个词的概率*, 其目标函数为:
$$ L(\theta) = \sum_tlogP(w_t|w_{t-n}, w_{t-n+1},\dots,w_{t-1})$$
使用长度为n-1的滑动窗口遍历整个语料库求和, 使得目标概率最大化,计算量正比于语料库大小,且预测的所有词概率总和应为1.
$$ \sum_{w\in\{vocabulary\}}P(w|w_{t-n+1},\dots,w_{t-1}) $$
NNLM 网络结构如下:  
<img src="./imgs/NNLM.jpg" width="480" height="320" align="center"/>  
图中Matrix C称为投影矩阵,把one-hot映射到稠密向量得到$C(W_{t-n+1}),\cdots,C(W_{t-1})$
隐藏层将稠密向量进行拼接,并使用tanh作为激活函数
隐层后面连接全连接层,使用softmax作为激活函数,预测词表中每个词出现的概率.
*特点*: 主要目的是为了搭建模型用于预测下一个可能的词, 词向量只是副产物. softmax的计算量太大.
#####  word2vec
word2ec有两种语言模型CBOW和Skip-gram他们的结构如下图所示:
<img src="./imgs/word2vec.png" width="480" height="320" align="center"/> 
具体的CBOW如下:  
<img src="./imgs/cbow.png" width="480" height="320" align="center"/> 
Skip-gram如下:  
<img src="./imgs/skip-gram.jpg" width="480" height="320" align="center"/> 
目标函数可简要概括为$$ J = \sum_{w\in corpus} P(w|context(w)) $$

和NNLM相比, word2vec:  
- 取消了隐藏层, 减少了计算量  
- 采用上下文滑窗而不是前文滑窗  
- 投影层使用简单的求和平均,而不是拼接向量  
可以看到，取消隐藏层，投影层求和平均都可以一定程度上减少计算量，但输出层的数量在那里，比如语料库有500W个词那么隐藏层就要对500W个神经元进行全连接计算，这依然需要庞大的计算量。因此,word2vec又提出了两种优化方法:  
- 使用hierarchical softmax和Negative sampling对输出层进行优化  
- hierarchical softmax  
    霍夫曼树是一棵特殊的二叉树，了解霍夫曼树之前先给出几个定义：
    路径长度：在二叉树路径上的分支数目，其等于路径上结点数-1  
    结点的权：给树的每个结点赋予一个非负的值
    结点的带权路径长度：根结点到该结点之间的路径长度与该节点权的乘积 
    树的带权路径长度：所有叶子节点的带权路径长度之和
    霍夫曼树的定义为：在权为$w_1,w_2,\cdots,w_n$的$n$个叶子结点所构成的所有二叉树中，带权路径长度最小的二叉树称为最优二叉树或霍夫曼树, 可以看出，结点的权越小，其离树的根结点越远。
word2vec算法利用霍夫曼树，将平铺型softmax压缩成层级softmax，不再使用全连接。具体做法是根据文本的词频统计，将词频赋给结点的权。在霍夫曼树中，叶子结点是待预测的所有词，在每个子结点处，用sigmoid激活后得到往左走的概率p，往右走的概率为1-p。最终训练的目标是最大化叶子结点处预测词的概率。    
- Negative sampling  
 负例采样的想法比较简单，假如有m个待预测的词，每次预测一个正样本词，其他的m-1个词均为负样本。一方面正负样本数差别太大，另一方面负样本中可能有很多不常用或者的词预测时概率基本为0，我们不想在计算它们的概率上面消耗资源。  
比如现在待预测词有100W个，正常情况下我们分类的全连接层需要100W个神经元，我们可以根据词语的出现频率进行负例采样，一个正样本加上采样出的比如说999个负样本，组成1000个新的分类全连接层。  
采样尽量保持了跟原样本一样的分布，具体做法是将$[0,1]$区间均分108份，然后根据词出现在语料库中的次数赋予每个词不同的份额。
$$len(w) = \frac{counter(w)}{\sum_{u \in D}counter(u)}$$
然后在$[0,1]$区间掷骰子，落在哪个区间内就采样哪个样本。实际上，最终效果证明上式中取$counter(w)$的$\frac{3}{4}$次方效果最好，所以在应用中也是这么做的。
##### fastText  
fasttext的模型与CBOW类似，实际上，fasttext的确是由CBOW演变而来的。CBOW预测上下文的中间词，fasttext预测文本标签。与word2vec算法的衍生物相同，稠密词向量也是在训练神经网络的过程中得到的。fasttext的结构图如下:
<img src="./imgs/fasttext.png" width="480" height="320" align="center"/>
fasttext的输入是一段词的序列，即一篇文章或一句话，输出是这段词序列属于某个类别的概率，所以fasttext是用来做文本分类任务的.
fasttext中采用层级softmax做分类，这与CBOW相同。fasttext算法中还考虑了词的顺序问题，即采用N-gram，与之前介绍离散表示时的做法相同。如：今天天气非常不错，Bi-gram的表示就是：今天、天天、天气、气非、非常、常不、不错。fasttext做文本分类对文本的存储方式有要求：
\__label__1, It is a nice day.
\__label__2, I am fine,thank you.
\__label__3, I like play football.

其中的__label__为实际类别的前缀，也可以自己定义。fasttext有python实现：https://github.com/salestock/fastText.py
```python
classifier = fasttext.supervised(input_file, output, label_prefix='__label__')
result = classifier.test(test_file)
print(result.precision,result.recall)
```
其中，input_file是已经按上面的格式要求做好的训练集txt，output后缀为.model，是我们保存的二进制文件，label_prefix可以自定义我们的类别前缀。
##### glove  
#### 动态表征  
##### ELMO  
##### GPT  
##### bert  